Важный этап в обучении модели препроцессинг данных. В задании сказано, что некоторые данные, как в тестовых, так и в тренировочных могут быть пропущенны некоторые значения. Эти значения нужно исключить или преобразовать в датасете.

In [ ]:
!unzip /content/skill-task2.zip

Archive:  /content/skill-task2.zip
  inflating: sample_submission.csv   
  inflating: test.csv/test.csv       
  inflating: train.csv/train.csv     


Подключаем библиотеки.

In [ ]:
import numpy as np
import pandas as pd

Оценка тренировочных данных.

In [ ]:
train_data = pd.read_csv('/content/train.csv/train.csv')

In [ ]:
train_data.head()

,sample_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,...,f1573,f1574,f1575,f1576,f1577,f1578,f1579,f1580,f1581,f1582,f1583,f1584,f1585,f1586,f1587,f1588,f1589,f1590,f1591,f1592,f1593,f1594,f1595,f1596,f1597,f1598,f1599,f1600,f1601,f1602,f1603,f1604,f1605,f1606,f1607,f1608,f1609,f1610,f1611,y
0,sample_0,25.609375,6.703125,3.652344,10.039062,169.375,102.8125,1.422852,6.722656,8.015625,8.109375,217.875,26.359375,6.562500,4.480469,8.617188,150.625,101.1250,1.770508,6.691406,3.468750,7.925781,266.750,22.562500,6.464844,4.066406,7.546875,158.625,85.6250,1.493164,6.558594,3.705078,7.890625,221.500,21.375000,6.527344,3.552734,7.160156,164.250,80.93750,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,7.0,2.0,2.0,28.0,0.0,4.0,3.0,0.0,4.0,3.0,0.0,0.0,2.0,1.0,2.0,3.675781,8.070312,4.363281,5.019531,5.710938,6.343750,6.843750,7.289062,7.617188,7.980469,1.0
1,sample_1,18.343750,5.824219,2.966797,4.902344,164.625,71.8125,1.357422,5.894531,2.753906,7.406250,192.875,21.703125,6.191406,3.722656,4.953125,147.625,80.6875,1.708008,6.328125,2.441406,7.621094,241.750,17.015625,6.066406,3.355469,3.798828,158.000,61.4375,1.429688,6.164062,2.185547,7.582031,195.750,13.937500,5.843750,2.833984,2.890625,159.625,48.96875,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,3.0,1.0,2.0,28.0,0.0,3.0,2.0,0.0,4.0,4.0,0.0,0.0,0.0,1.0,2.0,3.636719,7.359375,4.195312,4.808594,5.425781,5.949219,6.339844,6.730469,7.074219,7.175781,1.0
2,sample_2,28.562500,6.230469,3.583984,7.882812,159.500,113.1875,1.696289,6.316406,4.605469,7.753906,239.375,31.625000,6.378906,4.175781,6.910156,145.250,121.9375,1.989258,6.546875,2.892578,7.839844,284.250,24.921875,6.398438,3.964844,6.867188,150.250,93.7500,1.734375,6.527344,3.843750,7.871094,247.125,21.156250,6.355469,3.554688,5.332031,155.875,78.00000,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,20.0,4.0,2.0,1.0,28.0,0.0,3.0,3.0,0.0,4.0,5.0,1.0,0.0,0.0,1.0,4.0,3.806641,8.562500,4.523438,5.097656,5.789062,6.457031,6.871094,7.386719,7.878906,8.328125,1.0
3,sample_3,28.062500,6.132812,2.726562,6.378906,169.750,111.0000,1.535156,6.199219,3.712891,7.664062,205.750,27.187500,6.464844,3.212891,6.050781,155.250,102.9375,1.749023,6.589844,3.259766,7.886719,241.875,20.859375,6.226562,2.873047,4.144531,164.125,75.7500,1.468750,6.316406,2.730469,7.769531,188.500,19.546875,6.167969,2.576172,2.277344,165.000,70.12500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,4.0,3.0,1.0,17.0,0.0,6.0,0.0,0.0,1.0,5.0,1.0,0.0,0.0,1.0,1.0,3.156250,4.558594,3.533203,3.900391,4.261719,4.042969,3.869141,3.890625,4.042969,4.273438,1.0
4,sample_4,20.109375,6.144531,3.203125,6.035156,164.750,78.8750,1.281250,6.187500,4.003906,7.734375,187.875,22.640625,6.355469,4.121094,5.296875,145.750,84.4375,1.675781,6.488281,2.621094,7.816406,244.750,19.140625,6.292969,3.695312,4.156250,153.250,70.0625,1.421875,6.386719,2.613281,7.820312,203.125,15.562500,6.207031,3.042969,2.929688,158.125,54.78125,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,13.0,5.0,4.0,54.0,0.0,0.0,13.0,0.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,4.160156,6.613281,4.625000,4.996094,5.328125,5.593750,5.800781,6.027344,6.242188,6.449219,0.0


Удалим столбец 'ID'

In [ ]:
#train_data.drop(train_data.columns[0], axis=1, inplace=True)

Посмотрим на пропущенные данные.

**Размер тренировочного датасета (1095, 1613)**

Просуммируем количество пропущенных значений в столбцах и выведем в отдельный список.

In [ ]:
empty_train = []
for c in train_data:
    missing = train_data[c].isnull().sum()
    if missing > 0:
        row = c, missing
        empty_train.append(row)
del missing, row

Оценка тестовых данных.

In [ ]:
test_data = pd.read_csv('/content/test.csv/test.csv')

Удаляем ID

In [ ]:
#test_data.drop(test_data.columns[0], axis=1, inplace=True)

In [ ]:
test_data.head()

,sample_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,...,f1572,f1573,f1574,f1575,f1576,f1577,f1578,f1579,f1580,f1581,f1582,f1583,f1584,f1585,f1586,f1587,f1588,f1589,f1590,f1591,f1592,f1593,f1594,f1595,f1596,f1597,f1598,f1599,f1600,f1601,f1602,f1603,f1604,f1605,f1606,f1607,f1608,f1609,f1610,f1611
0,sample_1095,37.968750,6.757812,3.636719,10.789062,160.750,151.1250,1.791992,6.820312,8.140625,8.273438,248.125,33.531250,6.601562,4.347656,8.765625,145.750,130.2500,2.011719,6.757812,3.630859,8.039062,288.500,29.796875,6.605469,3.832031,6.984375,153.500,112.8125,1.752930,6.722656,4.218750,8.140625,240.250,27.937500,6.621094,3.548828,5.679688,155.625,104.56250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,9.0,1.0,3.0,28.0,0.0,3.0,6.0,0.0,3.0,7.0,2.0,0.0,0.0,1.0,1.0,3.664062,5.660156,4.152344,4.539062,4.953125,5.351562,5.312500,5.562500,5.644531,5.714844
1,sample_1096,23.859375,5.933594,3.019531,5.070312,164.750,94.0000,1.475586,6.007812,3.851562,7.527344,199.375,27.750000,6.218750,3.724609,4.000000,148.000,105.0000,1.830078,6.378906,2.236328,7.714844,250.625,19.640625,6.132812,3.271484,3.958984,156.625,71.5000,1.531250,6.238281,2.912109,7.664062,204.125,14.109375,5.839844,2.708984,2.087891,158.500,48.46875,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,2.0,3.0,2.0,23.0,0.0,3.0,2.0,0.0,3.0,5.0,1.0,0.0,0.0,1.0,2.0,3.496094,7.058594,4.132812,4.601562,5.160156,5.691406,6.054688,6.457031,6.875000,6.937500
2,sample_1097,40.250000,7.425781,3.734375,13.164062,172.125,161.7500,1.603516,7.410156,10.117188,8.804688,232.750,35.281250,7.062500,4.550781,10.179688,154.375,137.7500,1.801758,7.171875,4.058594,8.437500,271.750,32.343750,7.203125,3.919922,8.773438,165.000,124.4375,1.496094,7.257812,4.996094,8.671875,224.125,30.515625,7.011719,3.775391,7.730469,160.625,116.87500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,2.0,3.0,32.0,0.0,6.0,7.0,0.0,3.0,11.0,2.0,0.0,2.0,1.0,1.0,3.726562,6.398438,4.222656,4.687500,5.117188,5.332031,5.503906,5.796875,6.007812,6.199219
3,sample_1098,17.687500,5.761719,2.755859,4.390625,165.000,69.1875,1.344727,5.832031,3.070312,7.351562,189.500,20.625000,6.128906,3.404297,4.429688,148.375,76.0000,1.705078,6.269531,2.511719,7.570312,238.000,16.109375,5.937500,3.089844,3.154297,157.125,57.9375,1.415039,6.039062,2.220703,7.460938,189.625,13.492188,5.851562,2.597656,2.242188,159.750,46.81250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,1.0,2.0,1.0,18.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,3.177734,4.855469,3.675781,4.105469,4.515625,4.875000,4.769531,4.726562,4.855469,4.960938
4,sample_1099,23.828125,6.394531,3.140625,8.718750,163.250,94.1250,1.435547,6.445312,5.835938,7.902344,217.000,25.234375,6.472656,3.699219,7.539062,147.750,95.8125,1.735352,6.605469,3.482422,7.878906,259.250,21.062500,6.433594,3.400391,5.890625,157.125,78.5000,1.463867,6.515625,3.656250,7.914062,213.625,19.437500,6.425781,3.144531,4.933594,156.875,70.68750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,8.0,3.0,0.0,43.0,0.0,4.0,8.0,0.0,3.0,14.0,0.0,0.0,0.0,0.0,3.0,4.078125,6.582031,4.601562,5.031250,5.500000,5.980469,5.996094,6.179688,6.363281,6.480469


**Размер тестового датасета (194, 1612)**

Количество пропущенных значений в столбцах для тестовых данных.

In [ ]:
empty_test = []
for c in test_data:
    missing = test_data[c].isnull().sum()
    if missing > 0:
        row = c, missing
        empty_test.append(row)
del missing, row

Идея объединить данные о пропущенных столбцах вместе и посмотреть, где не сходятся названия (там будет вместо значений NaN)

In [ ]:
general_empty = pd.merge(pd.DataFrame(empty_test), pd.DataFrame(empty_train), on=0)

In [ ]:
for i in range(len(general_empty)):
    if general_empty['1_y'][i] is None or general_empty['1_x'][i] is None: 
        print(i)

Получается, столбцы у которых пропущены значения в тренировочных и тестовых данных, совпадают!

Тогда необходимо определиться в каком соотношение удалять столбцы и строки. Задача, оставить наибольшее количество данных (но не все).

Посчитаем количество строк для тестовых и тренировочных данных в размере 5%.

In [ ]:
int(1095 * 0.05), int(194 * 0.05)

(54, 9)

Получаем порог среза столбцов. 

In [ ]:
general_empty[general_empty['1_y'] > 54].shape

(174, 3)

In [ ]:
general_empty[general_empty['1_x'] > 9].shape

(174, 3)

Интересно, что количество строк опять совпадают (количество строк в этом случае - это название столбцов).

**Срезаем данные для тренировочного датасета.**

Получим название столбцов для среза.

In [ ]:
empty_name_train = general_empty[general_empty['1_y'] > 54][0].tolist()

Удаляем столбцы.

In [ ]:
train_data = train_data.drop(columns=empty_name_train)

Удаляем строки с NaN.

In [ ]:
train_data = train_data.dropna()

Проверяем наличие NaN.

In [ ]:
for c in train_data:
    no_missing = train_data[c].isnull().sum()
    if no_missing > 0:
        print(no_missing)

**Все ячейки имеют значения.**

Получим название столбцов для среза.


In [ ]:
empty_name_test = general_empty[general_empty['1_x'] > 9][0].tolist()

Удаляем столбцы.

In [ ]:
test_data = test_data.drop(columns=empty_name_test)

Удаляем строки с NaN.

In [ ]:
test_data = test_data.dropna()

Проверяем наличие NaN.

In [ ]:
for c in test_data:
    no_missing = test_data[c].isnull().sum()
    if no_missing > 0:
        print(no_missing)

Размер тренировочного и тестового датасета.

In [ ]:
print(train_data.shape, test_data.shape) #in train data is y (prediction), test data without it

(1017, 1440) (176, 1439)


**Оценим количество срезанных. **

P.S. В тренировочном датасете, есть столбец Y для предсказания. Таким образом количество столбцов совпадает.

Процент среза по столбцам для тренировочного, тестового, для обоих по колонкам.

In [ ]:
(1017 / 1095) * 100, ((1440 - 2) / 1612) * 100, (176 / 194) * 100

(92.87671232876711, 89.20595533498759, 90.72164948453609)

Для строк тренировочных данных, удалили 7,2%.
Для строк тестовых данных удалили 9,3%.
Для столбцов удалили 10,8% данных.

Сравним, не похожие столбцы.

In [ ]:
col1 = train_data.drop(columns=['y']).columns

In [ ]:
col2 = test_data.columns

In [ ]:
for i in range(len(col1)):
    if col1[i] != col2[i]:
        print(i)

Значит все имена столбцов совпадают!

*Идея, если колонка не содержит 0, и при этом большое значение NaN, то может более ликвидно не удалять их, а значение NaN заменять на 0, тогда мы оставим большое колличество данных. Но в этом случае, машина будет считать, что значения NaN (как координаты точек) равным между собой, но они не могут равнятся, т.к. это подобие идеи бесконечности.*


Удаление +- inf

In [ ]:
for c in test_data:
    no_missing = test_data.loc[test_data[c] == np.inf ,c].sum()
    if no_missing > 0:
        print(c)
        print(test_data[test_data[c] == np.inf])

f267
       sample_id         f0        f1  ...     f1609     f1610     f1611
53   sample_1158  19.187500  6.023438  ...  5.851562  6.023438  6.136719
111  sample_1220  20.015625  6.199219  ...  6.187500  6.421875  6.648438

[2 rows x 1439 columns]
f273
       sample_id         f0        f1  ...     f1609     f1610     f1611
52   sample_1157  21.968750  6.195312  ...  7.800781  8.273438  8.656250
53   sample_1158  19.187500  6.023438  ...  5.851562  6.023438  6.136719
111  sample_1220  20.015625  6.199219  ...  6.187500  6.421875  6.648438
132  sample_1242  21.453125  6.109375  ...  7.808594  8.281250  8.679688

[4 rows x 1439 columns]
f278
       sample_id         f0        f1  ...     f1609     f1610     f1611
52   sample_1157  21.968750  6.195312  ...  7.800781  8.273438  8.656250
53   sample_1158  19.187500  6.023438  ...  5.851562  6.023438  6.136719
100  sample_1208  21.125000  6.167969  ...  7.460938  7.890625  8.273438
111  sample_1220  20.015625  6.199219  ...  6.187500  6.421

In [ ]:
for c in train_data:
    no_missing = train_data.loc[train_data[c] == np.inf ,c].sum()
    if no_missing > 0:
        print(c)
        print(train_data[train_data[c] == np.inf])

f267
      sample_id         f0       f1  ...     f1610     f1611    y
496  sample_533  20.671875  6.03125  ...  6.027344  6.144531  1.0

[1 rows x 1440 columns]
f273
        sample_id         f0        f1  ...     f1610     f1611    y
17      sample_17  21.562500  6.042969  ...  8.234375  8.609375  1.0
78      sample_83  20.031250  6.089844  ...  6.183594  6.304688  1.0
90      sample_96  21.250000  6.121094  ...  8.234375  8.609375  1.0
175    sample_189  21.234375  6.089844  ...  8.250000  8.640625  1.0
247    sample_265  20.468750  6.199219  ...  6.289062  6.492188  1.0
327    sample_349  20.687500  6.058594  ...  8.203125  8.578125  1.0
496    sample_533  20.671875  6.031250  ...  6.027344  6.144531  1.0
553    sample_594  20.812500  6.074219  ...  6.929688  7.230469  1.0
719    sample_770  23.343750  6.457031  ...  7.066406  7.359375  1.0
748    sample_801  24.140625  6.437500  ...  7.011719  7.328125  1.0
760    sample_813  25.171875  6.328125  ...  7.050781  7.367188  1.0
782  

Попытаемся сохранить баланс.

In [ ]:
train_data[~train_data.isin([np.nan, np.inf, -np.inf]).any(1)].shape

(964, 1440)

In [ ]:
test_data[~test_data.isin([np.nan, np.inf, -np.inf]).any(1)].shape

(164, 1439)

In [ ]:
(964 / 1095) * 100, ((1440 - 2) / 1612) * 100, (164 / 194) * 100

(88.03652968036529, 89.20595533498759, 84.5360824742268)

В сравнении с первоначальным датасетом, при удалении -+inf (бесконечность).

Для строк тренировочных данных, останется 12%. Для строк тестовых данных останется 15,5%. Для столбцов останется 10,8% данных.

Можно сохранить баланс, удалив столбцы с большим количеством строк, при этом средняя потеря по строкам и столбцам тестового и тренировочного датасета составит 10-12%. Но, тренировочный датасет не так важен для обучения, как длина вектора. Оставим значения столбцов, удалив строки.

In [ ]:
train_data = train_data[~train_data.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
test_data = test_data[~test_data.isin([np.nan, np.inf, -np.inf]).any(1)]

Сохраним обработанные данные.

In [ ]:
train_data.to_csv('train_p.csv', index=False) #index false

In [ ]:
test_data.to_csv('test_p.csv', index=False) #index false

In [ ]:
with open("col_name.txt", "w") as f:
    f.writelines(str(empty_name_test))